In [1]:
import pandas as pd 
import scsims
from sklearn.preprocessing import LabelEncoder
import sys
import anndata as an
import scanpy as sp
import h5py

sys.path.append('../src')
sys.path.append('../tests')

from models.lib.lightning_train import DataModule, generate_trainer
from models.lib.data import *
from models.lib.neural import *
from pytorch_lightning.loggers import WandbLogger
from torchmetrics.functional import *
import pandas as pd
from scipy.sparse import csr_matrix
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [2]:
urls = {
    'developing_mouse_brain.tsv':[
        'https://cells.ucsc.edu/mouse-dev-brain/exprMatrix.tsv.gz',
        'https://cells.ucsc.edu/mouse-dev-brain/meta.tsv',
    ]
}

module = DataModule(
    urls=urls,
    class_label='Class',
    assume_numeric_label=False,
    batch_size=16,
    num_workers=0,
    sep='\t',
    datapath='./developing_mouse_brain/'
)

# module.prepare_data()
module.setup()

Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


In [3]:
loader = module.trainloader

next(iter(loader))

ValueError: could not convert string to float: 'ENSMUSG00000030324|Rho'

In [5]:
!ls developing_mouse_brain/

a                                 r
developing_mouse_brain.tsv        w
labels_developing_mouse_brain.tsv


In [10]:
pd.read_csv('developing_mouse_brain/developing_mouse_brain.tsv', sep='\t', nrows=1)

,gene,10X66_4_A_1:CATTTCGAGATACCx,10X66_4_A_1:ACGTTGGATTTGGGx,10X66_4_A_1:TATGTCTGGTAAAGx,10X68_6_A_1:CCTGACTGAGTCACx,10X68_6_A_1:AGTATAACGAACCTx,10X66_4_A_1:TAGCTACTCTAGTGx,10X66_4_A_1:AGAAGATGAGGTTCx,10X66_4_A_1:TGTAGTCTGGTTTGx,10X66_4_A_1:TCCACGTGGAAACAx,...,10X68_4_A_1:GGTTGAACGCCATAx,10X68_4_A_1:AAATCCCTTAACGCx,10X70_6_A_1:ACCTCCGACGTTGAx,10X70_3_A_1:CTTCACCTATTCGGx,10X70_6_A_1:CCTAGAGATGGAGGx,10X70_3_A_1:GTCGAATGGGTGTTx,10X70_3_A_1:AGGTCATGCAAGCTx,10X70_6_A_1:GCACAATGGATGAAx,10X70_3_A_1:CATCAGGAGTGAGGx,10X70_6_A_1:ATTGGTCTAACCACx
0,ENSMUSG00000018474|Chd3,2,4,17,7,2,1,8,0,5,...,4,3,2,6,0,3,1,3,5,4


In [13]:
from bigcsv.bigcsv import experimental_to_h5ad

experimental_to_h5ad(
    file='./developing_mouse_brain/developing_mouse_brain.tsv',
    outfile='./developing_mouse_brain/developing_mouse_brain.h5ad',
    sep='\t',
    index_col='gene',
    chunksize=100,
    sparsify=True,
)

Working on chunk 0/310


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Working on chunk 1/310


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Working on chunk 2/310


...

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Working on chunk 310/310


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Concatenating h5ad's


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [15]:
data = an.read_h5ad('./developing_mouse_brain/developing_mouse_brain.h5ad')

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [21]:
data.obs = data.obs.reset_index(drop=True)
data = data.T
data

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 292495 × 31053
    var: 'gene'

In [ ]:
data.write_h5ad('../data/developing_mouse_brain/')

In [9]:
tabnetmodel = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
        'weight_decay': 0,
    },
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau,
        'factor': 0.001,
    },
    metrics={
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    },
)

wandb_logger = WandbLogger(
    project=f"Retina Model",
    name='local-retina-model'
)

early_stop = EarlyStopping(
    monitor="weighted_val_accuracy", 
    min_delta=0.00, 
    patience=3,
    verbose=False, 
    mode="max"
)

trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=[early_stop, lr_monitor, checkpoint],
    max_epochs=100,
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/julian/Documents/Projects/organoid-classification/notebooks/../src/models/lib/../../../data/raw/developing_mouse_brain.tsv'